In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("./openpowerlifting.csv")
data.drop(["Squat4Kg","Bench4Kg","Deadlift4Kg"],axis=1,inplace=True)
data.drop_duplicates(inplace=True)
display(data.head())
display(data.describe())

In [ ]:
data_cleaned = data[(data['BestSquatKg'] > 0)&(data['BestBenchKg']>0)&(data['BestDeadliftKg']> 0)]
# encode
data_cleaned=data_cleaned.dropna()
data_cleaned['Sex'] = data_cleaned['Sex'].map( {'M':1, 'F':0} )
# encode
from sklearn.preprocessing import LabelEncoder
data_cleaned['Equipment'] = LabelEncoder().fit_transform(data_cleaned['Equipment'])
data_cleaned['WeightClassKg'] = LabelEncoder().fit_transform(data_cleaned['WeightClassKg'])

data_cleaned.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.pairplot(data_cleaned[['Age','BodyweightKg','Wilks','BestSquatKg','BestBenchKg','BestDeadliftKg']])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns;sns.set();
fig,ax = plt.subplots(1,3)
fig.set_size_inches(20, 15)
sns.distplot(data_cleaned['BestSquatKg'],ax=ax[0])
sns.distplot(data_cleaned['BestSquatKg'],ax=ax[1])
sns.distplot(data_cleaned['BestSquatKg'],ax=ax[2])

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from keras.activations import *
from keras.optimizers import *
from keras.models import *
from keras.layers import *
#feature_names = ['Age','BodyweightKg', 'Equipment','Wilks','WeightClassKg','Sex']
#outcome_names = ['BestSquatKg','BestBenchKg','BestDeadliftKg']

def fit_with_features(features,outcomes):
    # features
    X = data_cleaned[features]
    # rescale features
    scaler_X = MinMaxScaler(feature_range=(0, 1)) 
    #X = scaler_X.fit_transform(X)
    # outcomes
    y = data_cleaned[outcomes]
    # split into a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    # create model
    model = Sequential()
    model.add(Dense(np.power(2, 5), input_dim=X_train.shape[1], activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer= 'adam',
                  loss="mean_squared_error",
                  metrics=['acc'])
    result = model.fit(X_train, y_train,batch_size=32,epochs=3,verbose=1)
    evaluation = model.evaluate(X_test,y_test)
    print("Accuracy:",evaluation[1])

In [ ]:
fit_with_features(['Age','BodyweightKg', 'Equipment','Wilks','WeightClassKg','Sex'],['BestSquatKg','BestBenchKg','BestDeadliftKg'])
fit_with_features(['Age','BodyweightKg', 'Equipment','WeightClassKg','Sex'],['BestSquatKg','BestBenchKg','BestDeadliftKg'])
fit_with_features(['Age','BodyweightKg', 'Equipment','Wilks','Sex'],['BestSquatKg','BestBenchKg','BestDeadliftKg'])
fit_with_features(['Age','BodyweightKg','Sex'],['BestSquatKg','BestBenchKg','BestDeadliftKg'])

In [ ]:
data_grouped = pd.concat([data_cleaned.groupby(['WeightClassKg'])['BestSquatKg'].mean()
                            ,data_cleaned.groupby(['WeightClassKg'])['BestBenchKg'].mean()
                            ,data_cleaned.groupby(['WeightClassKg'])['BestDeadliftKg'].mean()],axis=1)
data_grouped.columns = ['MeanBestSquatKg','MeanBestBenchKg','MeanBestDeadliftKg']